In [113]:
# Importing used libraries
import pandas as pd
from collections import defaultdict
from statistics import mean
import json

In [114]:
# Reading JSON file into a pandas DataFrame
df = pd.read_json("Order_Export.json", lines=True)

In [115]:
# Inspecting DataFrame Structure
df.head(5)

,_id,client,items
0,126531,"{'id': 56468, 'name': 'DROG. SAO JUDAS', 'raza...","[{'product': {'sku': 'P00032604', 'name': 'VIT..."
1,126517,"{'id': 56638, 'name': 'DROG. COTIFARMA', 'raza...","[{'product': {'sku': 'P00014022', 'name': 'EXP..."
2,126454,"{'id': 57821, 'name': 'FARMAIS', 'razaoSocial'...","[{'product': {'sku': 'P00028545', 'name': 'VIT..."
3,126449,"{'id': 63631, 'name': 'E+FARMA ITAPEVA', 'raza...","[{'product': {'sku': 'P00028541', 'name': 'ASS..."
4,126448,"{'id': 64402, 'name': 'BRASIL GENERICOS', 'raz...","[{'product': {'sku': 'P00020620', 'name': 'NIM..."


In [116]:
# Inspecting client column structure
df.iloc[0].loc['client']

{'id': 56468,
 'name': 'DROG. SAO JUDAS',
 'razaoSocial': 'DROGARIA SAO JUDAS DE MAUA LTDA - ME'}

In [117]:
# Inspecting items column structure
df.iloc[0].loc['items']

[{'product': {'sku': 'P00032604',
   'name': 'VITAXON C 1G C/30CPR EFERV-LARANJ (VIT.C',
   'maker': 'AIRELA'},
  'quantity': 10,
  'averagePrice': 17.36,
  'distributorBuyingPrice': 8.31,
  'price': 13.89},
 {'product': {'sku': 'P00031400',
   'name': 'VITACIN 1G C/10ENV EFE 4G-LAR (VIT.C',
   'maker': 'GEOLAB'},
  'quantity': 30,
  'averagePrice': 5.84,
  'distributorBuyingPrice': 3.31,
  'price': 4.86},
 {'product': {'sku': 'P00025011',
   'name': 'BIOVITA C 1G C/50ENV EFE S/AC-LAR (VIT.C',
   'maker': 'VITAMED'},
  'quantity': 21,
  'averagePrice': 35.65,
  'distributorBuyingPrice': 18.8,
  'price': 31.33}]

In [118]:
# Creating aggregating variables
margin_by_product = defaultdict(list)
margin_by_maker = defaultdict(list)
client_orders = defaultdict(list)
total = 0
min_number_of_itens = None
max_number_of_itens = None
total_num_of_itens = 0
num_of_orders = 0
total_global_margin = []

In [119]:
# Aggregating values for each record, its client and each of its items
for record in range(len(df)):
    client_order = 0
    num_of_orders += 1
    num_of_itens = len(df.iloc[record].loc['items'])
    total_num_of_itens += num_of_itens
    min_number_of_itens = num_of_itens if (min_number_of_itens is None or min_number_of_itens > num_of_itens) else min_number_of_itens
    max_number_of_itens = num_of_itens if (max_number_of_itens is None or max_number_of_itens < num_of_itens) else max_number_of_itens
    for product in df.iloc[record].loc['items']:
        product_data = product.get('product')
        sku = product_data.get('sku')
        name = product_data.get('name')
        maker = product_data.get('maker')
        quantity = product.get('quantity')
        price = product.get('price')
        distributorBuyingPrice = product.get('distributorBuyingPrice')
        total += price * quantity
        margin = (price - distributorBuyingPrice) / distributorBuyingPrice
        total_global_margin.append(margin)
        margin_by_product[f'{sku};{name}'].append(margin)
        margin_by_maker[maker].append(margin)
        client_order += price * quantity
    client_data = df.iloc[record].loc['client']
    client_id = client_data.get('id')
    client_name = client_data.get('name')
    client_orders[f'{client_id};{client_name}'].append(client_order)

In [120]:
# Structuring product margins information
productMargins = []
for sku_name, margins in margin_by_product.items():
    sku, name = sku_name.split(';')
    productMargin = {
        'sku': sku,
        'name': name,
        'margin': mean(margins)
    }
    productMargins.append(productMargin)

In [121]:
# Structuring maker margins information
makerMargins = []
for maker, margins in margin_by_maker.items():
    makerMargin = {
        'maker': maker,
        'margin': mean(margins)
    }
    makerMargins.append(makerMargin)

In [122]:
# Structuring client stats information
clientStats = []
for client_name, orders in client_orders.items():
    client_id_, name = client_name.split(';')
    singleClientStats = {
        'id': client_id,
        'name': name,
        'orderCount': len(orders),
        'orderTotal': sum(orders)
    }
    clientStats.append(singleClientStats)

In [123]:
# Structuring result JSON
result = {
    'salesTotal': total,
    'averageOrder': total / num_of_orders,
    'averageOrderItems': total_num_of_itens / num_of_orders,
    'minOrderItens': min_number_of_itens,
    'maxOrderItens': max_number_of_itens,
    'globalMargin': mean(total_global_margin),
    'productMargins': productMargins,
    'makerMargins': makerMargins,
    'clientStats': clientStats
}

In [125]:
# Writing result JSON to a file
with open('result_orders.json', 'w') as result_file:
    json.dump(result, result_file)

In [124]:
# Printing result JSON
print(json.dumps(result, indent=2))

{
  "salesTotal": 1289421.6299999084,
  "averageOrder": 1289.4216299999084,
  "averageOrderItems": 17.704,
  "minOrderItens": 0,
  "maxOrderItens": 158,
  "globalMargin": 0.6847010815583214,
  "productMargins": [
    {
      "sku": "P00032604",
      "name": "VITAXON C 1G C/30CPR EFERV-LARANJ (VIT.C",
      "margin": 0.6714801444043321
    },
    {
      "sku": "P00031400",
      "name": "VITACIN 1G C/10ENV EFE 4G-LAR (VIT.C",
      "margin": 0.46827794561933545
    },
    {
      "sku": "P00025011",
      "name": "BIOVITA C 1G C/50ENV EFE S/AC-LAR (VIT.C",
      "margin": 0.6664893617021275
    },
    {
      "sku": "P00014022",
      "name": "EXPEC XPE 120ML (GUAI+IOD.POT+BENZ.SOD+O",
      "margin": 0.380103550295858
    },
    {
      "sku": "P00030475",
      "name": "DIGOXINA 0,25MG C/20CPR (GEN)",
      "margin": 0.7259615384615383
    },
    {
      "sku": "P00022787",
      "name": "ENXAK C/12CPR (DIPIRONA+CAF+DI-HIDROERG",
      "margin": 0.4747372845733501
    },
    {
     